In [28]:
!pip install nvidia-pyindex && pip install nvidia-cublas

  Using cached nvidia-pyindex-1.0.9.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8418 sha256=a76cc9cc94046724e8a64eef92cb8d00579d68264b649b646ad136a1537ee7b2
  Stored in directory: /home/ubuntu/.cache/pip/wheels/2c/af/d0/7a12f82cab69f65d51107f48bcd6179e29b9a69a90546332b3
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 MB 252.0 MB/s eta 0:00:0000:0100:01


In [1]:
# !pip install -r requirements.txt
# !pip install --force-reinstall --ignore-installed --no-cache-dir llama-cpp-python
import json
import boto3
import os
from os import environ
import time
from llama_cpp import Llama

model_path='/opt/dlami/nvme/firefly-llama2-13b-v1.2.Q2_K.gguf'
n_ctx=512

model = Llama(model_path=model_path, n_ctx=n_ctx, n_gpu_layers=-1, use_mlock=True,main_gpu=0)
model('how to learn english asdfhasklfhsilufhsialgvhsailkvhdkljvhvk;ZVhkjlVhJKLV ', max_tokens=512, temperature=0)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /opt/dlami/nvme/firefly-llama2-13b-v1.2.Q2_K.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q2_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q2_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q3_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q3_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q3_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_down.weight q3_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    7:              blk.0.ffn_up.weight q3_K     [  5120,

{'id': 'cmpl-a42d86e0-fed4-4857-a3ac-5fd573201f72',
 'object': 'text_completion',
 'created': 1695714208,
 'model': '/opt/dlami/nvme/firefly-llama2-13b-v1.2.Q2_K.gguf',
 'choices': [{'text': "\nI'm sorry, but I cannot understand your question. Please rephrase it or provide more context so that I can assist you better.",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 42, 'completion_tokens': 30, 'total_tokens': 72}}

In [21]:
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.session import Session

#ECR URI
image_uri = '969422986683.dkr.ecr.cn-northwest-1.amazonaws.com.cn/llama213bint4'
role = 'arn:aws-cn:iam::969422986683:role/AmazonSageMaker-ExecutionRole-20200517T121567'
# This can be dummy model file
model_dir = 's3://sagemaker-cn-northwest-1-969422986683/model.tar.gz'

# Create the SageMaker model instance
model = Model(
    image_uri=image_uri,
    role=role,
    model_data=model_dir
)
endpoint_name = 'llama2-cpp-test'
import boto3

client = boto3.client('sagemaker')
try:
    response = client.delete_endpoint_config(EndpointConfigName=endpoint_name)
except:
    pass

model.deploy(
    instance_type='ml.p3.2xlarge',
    initial_instance_count=1,
    endpoint_name = endpoint_name,
)

--------!

In [24]:
import boto3
import time

endpoint_name = 'llama2-cpp-test'
data = {"prompt": "how to learn english?"}
runtime_sagemaker_client = boto3.client(service_name="sagemaker-runtime")


body = json.dumps(data)

start = time.time()
response = runtime_sagemaker_client.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType  = "application/json",
    Body= body)

cost = time.time() - start     
result = response['Body'].read().decode('utf-8')

print('Response: ', result)
print("Cost Time:  %s seconds" % (cost))
print('Output Chars :', len(result))
print('Speed: {:.2f} Chars/s'.format(len(result)/float(cost)))

ReadTimeoutError: Read timeout on endpoint URL: "https://runtime.sagemaker.cn-northwest-1.amazonaws.com.cn/endpoints/llama2-cpp-test/invocations"